In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import timm
import albumentations as A
import cv2
from albumentations.pytorch import ToTensorV2
import torch

from datasetClass import KSLDataset
from LeNetClass import LeNet
from trainLoop import Trainer

# Загрузка данных

In [3]:
df = pd.read_csv(r"data/Train.csv")

In [5]:
df.groupby(by = "img_IDS").count().value_counts()

Label
1        6249
Name: count, dtype: int64

In [5]:
df

,img_IDS,Label
0,ImageID_33HMDMJ3,Temple
1,ImageID_V5POEX4O,Church
2,ImageID_89CCCCW6,Enough/Satisfied
3,ImageID_GNJXOWX5,Me
4,ImageID_7Q9LOP7R,Love
...,...,...
6244,ImageID_F2HTAA5P,Love
6245,ImageID_0D69ZQ4X,Church
6246,ImageID_E67IZXVJ,Seat
6247,ImageID_ENGL8NP3,You


In [9]:
df['Label'].value_counts()

Label
Enough/Satisfied    695
Seat                695
Mosque              695
Temple              694
Church              694
Love                694
Me                  694
You                 694
Friend              694
Name: count, dtype: int64

In [4]:
df["Label"] = df["Label"].astype("category")
df["target"] = df["Label"].cat.codes

In [26]:
df

,img_IDS,Label,target
0,ImageID_33HMDMJ3,Temple,7
1,ImageID_V5POEX4O,Church,0
2,ImageID_89CCCCW6,Enough/Satisfied,1
3,ImageID_GNJXOWX5,Me,4
4,ImageID_7Q9LOP7R,Love,3
...,...,...,...
6244,ImageID_F2HTAA5P,Love,3
6245,ImageID_0D69ZQ4X,Church,0
6246,ImageID_E67IZXVJ,Seat,6
6247,ImageID_ENGL8NP3,You,8


In [5]:
train_df, val_df = train_test_split(
    df,
    test_size=0.33, random_state=42, stratify=df["Label"])

# Загрузка изображений

In [6]:
images_dir = "data/Images/Images"

train_dataset = KSLDataset(train_df, images_dir)
val_dataset   = KSLDataset(val_df,   images_dir)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,  num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

# Запуск обучающего цикла LeNet

In [10]:
model = LeNet(in_channels=3,
              num_classes=len(train_df.target.unique()), use_dropout=True)

trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=10)

Epoch 1/10 | train_loss: 2.2005 | val_loss: 2.1977 | val_ROC_AUC: 0.5140
Epoch 2/10 | train_loss: 2.1987 | val_loss: 2.1984 | val_ROC_AUC: 0.5203
Epoch 3/10 | train_loss: 2.1990 | val_loss: 2.1961 | val_ROC_AUC: 0.6027
Epoch 4/10 | train_loss: 2.1795 | val_loss: 2.1400 | val_ROC_AUC: 0.6657
Epoch 5/10 | train_loss: 2.0240 | val_loss: 1.8765 | val_ROC_AUC: 0.7588
Epoch 6/10 | train_loss: 1.8370 | val_loss: 1.7496 | val_ROC_AUC: 0.7910
Epoch 7/10 | train_loss: 1.7244 | val_loss: 1.6758 | val_ROC_AUC: 0.8128
Epoch 8/10 | train_loss: 1.6012 | val_loss: 1.6342 | val_ROC_AUC: 0.8241
Epoch 9/10 | train_loss: 1.5188 | val_loss: 1.5785 | val_ROC_AUC: 0.8347
Epoch 10/10 | train_loss: 1.4429 | val_loss: 1.5252 | val_ROC_AUC: 0.8453
Best val ROC AUC: 0.8453


# Запуск обучающего цикла для resnet18

In [7]:
model = timm.create_model("resnet18", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_resnet18.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [8]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 0.9806 | val_loss: 0.4772 | val_ROC_AUC: 0.9852
Epoch 2/4 | train_loss: 0.3627 | val_loss: 0.3742 | val_ROC_AUC: 0.9902
Epoch 3/4 | train_loss: 0.2154 | val_loss: 0.4184 | val_ROC_AUC: 0.9892
Epoch 4/4 | train_loss: 0.1420 | val_loss: 0.4470 | val_ROC_AUC: 0.9879
Best val ROC AUC: 0.9902


# Запуск обучающего цикла для efficientnet_b0

In [9]:
model = timm.create_model("efficientnet_b0.ra_in1k", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_efficientnet_b0.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [11]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 0.2333 | val_loss: 0.3685 | val_ROC_AUC: 0.9908
Epoch 2/4 | train_loss: 0.1601 | val_loss: 0.3848 | val_ROC_AUC: 0.9923
Epoch 3/4 | train_loss: 0.0953 | val_loss: 0.4778 | val_ROC_AUC: 0.9898
Epoch 4/4 | train_loss: 0.1264 | val_loss: 0.4199 | val_ROC_AUC: 0.9911
Best val ROC AUC: 0.9923


# Запуск обучающего цикла для convnext_tiny

In [12]:
model = timm.create_model("convnext_tiny.in12k_ft_in1k", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_convnext_tiny.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [13]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 2.3140 | val_loss: 2.2472 | val_ROC_AUC: 0.5075
Epoch 2/4 | train_loss: 2.2196 | val_loss: 2.2164 | val_ROC_AUC: 0.4997
Epoch 3/4 | train_loss: 2.2075 | val_loss: 2.2084 | val_ROC_AUC: 0.5087
Epoch 4/4 | train_loss: 2.2039 | val_loss: 2.2000 | val_ROC_AUC: 0.5101
Best val ROC AUC: 0.5101


# Запуск обучающего цикла для densenet

In [14]:
model = timm.create_model("densenet121.ra_in1k", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_densenet121.ra_in1k.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [15]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 0.9194 | val_loss: 0.8185 | val_ROC_AUC: 0.9642
Epoch 2/4 | train_loss: 0.4744 | val_loss: 0.6755 | val_ROC_AUC: 0.9750
Epoch 3/4 | train_loss: 0.3612 | val_loss: 0.6173 | val_ROC_AUC: 0.9781
Epoch 4/4 | train_loss: 0.2884 | val_loss: 0.5608 | val_ROC_AUC: 0.9800
Best val ROC AUC: 0.9800


# Проверка разных аугментаций

In [17]:
augs = { "base":
    A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
]), # базовая, приведение к одинаковому масштабу
    "ratate" :A.Compose([
        A.Resize(224, 224),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(
            shift_limit=0.05,
            scale_limit=0.05,
            rotate_limit=10,
            p=0.5,
            border_mode=cv2.BORDER_REFLECT_101
        ),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]), # отражение+поворот, делает устойчивой к положению объекта
    "color" :A.Compose([
    A.Resize(224, 224),
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
]), # цвет+яркость, делает устойчивой к освещению/оттенкам
    "cut": A.Compose([
        A.Resize(224, 224),
        A.CoarseDropout(
            num_holes_range=(3, 6),
            hole_height_range=(10, 20),
            hole_width_range=(10, 20),
            fill="random_uniform",
            p=1.0),
        A.Normalize(mean=(0.485, 0.456, 0.406),
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]), # закрытие, делает устойчивой к паттернам 
    "mix" : A.Compose([
    A.Resize(224, 224),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(
        shift_limit=0.05,
        scale_limit=0.05,
        rotate_limit=10,
        p=0.5,
        border_mode=cv2.BORDER_REFLECT_101
    ),
    A.RandomBrightnessContrast(p=0.5),
    A.CoarseDropout(
        num_holes_range=(3, 6),
        hole_height_range=(10, 20),
        hole_width_range=(10, 20),
        fill="random_uniform",
        p=1.0
    ),
    A.Normalize(mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225)),
    ToTensorV2()
]) # комбинация
}

In [18]:
model = timm.create_model("efficientnet_b0.ra_in1k", pretrained=False)
state_dict = torch.load("model_weights/pytorch_model_efficientnet_b0.bin", map_location="cpu")
model.load_state_dict(state_dict)
model.reset_classifier(num_classes=len(train_df.target.unique()))

In [19]:
for key in augs:
    train_dataset = KSLDataset(train_df, images_dir, transform = augs[key])
    val_dataset   = KSLDataset(val_df,   images_dir, transform = augs["base"])

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,  num_workers=4, pin_memory=True)
    val_loader   = DataLoader(val_dataset,   batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

    print("-----", key, "-----")
    
    trainer = Trainer(model, train_loader, val_loader, lr=1e-3)
    trainer.fit(num_epochs=4)

----- base -----
Trainer device: cuda
Epoch 1/4 | train_loss: 0.7483 | val_loss: 0.3634 | val_ROC_AUC: 0.9908
Epoch 2/4 | train_loss: 0.3208 | val_loss: 0.3863 | val_ROC_AUC: 0.9900
Epoch 3/4 | train_loss: 0.2796 | val_loss: 0.3481 | val_ROC_AUC: 0.9913
Epoch 4/4 | train_loss: 0.1912 | val_loss: 0.3489 | val_ROC_AUC: 0.9930
Best val ROC AUC: 0.9930
----- ratate -----
Trainer device: cuda
Epoch 1/4 | train_loss: 0.3097 | val_loss: 0.3137 | val_ROC_AUC: 0.9935
Epoch 2/4 | train_loss: 0.2372 | val_loss: 0.3694 | val_ROC_AUC: 0.9912
Epoch 3/4 | train_loss: 0.2031 | val_loss: 0.3795 | val_ROC_AUC: 0.9915
Epoch 4/4 | train_loss: 0.1741 | val_loss: 0.3685 | val_ROC_AUC: 0.9924
Best val ROC AUC: 0.9935
----- color -----
Trainer device: cuda
Epoch 1/4 | train_loss: 0.1807 | val_loss: 0.3820 | val_ROC_AUC: 0.9919
Epoch 2/4 | train_loss: 0.1199 | val_loss: 0.3543 | val_ROC_AUC: 0.9929
Epoch 3/4 | train_loss: 0.0956 | val_loss: 0.4321 | val_ROC_AUC: 0.9913
Epoch 4/4 | train_loss: 0.1055 | val_loss

# MixUp/CutMix

In [20]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3, mix_mode="mixup")
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 0.5348 | val_loss: 0.3345 | val_ROC_AUC: 0.9899
Epoch 2/4 | train_loss: 0.4589 | val_loss: 0.3504 | val_ROC_AUC: 0.9873
Epoch 3/4 | train_loss: 0.5155 | val_loss: 0.3507 | val_ROC_AUC: 0.9885
Epoch 4/4 | train_loss: 0.4473 | val_loss: 0.3565 | val_ROC_AUC: 0.9895
Best val ROC AUC: 0.9899


In [21]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3, mix_mode="cutmix")
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 0.6603 | val_loss: 0.3391 | val_ROC_AUC: 0.9875
Epoch 2/4 | train_loss: 0.6615 | val_loss: 0.3284 | val_ROC_AUC: 0.9899
Epoch 3/4 | train_loss: 0.6567 | val_loss: 0.3892 | val_ROC_AUC: 0.9865
Epoch 4/4 | train_loss: 0.5921 | val_loss: 0.3162 | val_ROC_AUC: 0.9903
Best val ROC AUC: 0.9903


In [22]:
trainer = Trainer(model, train_loader, val_loader, lr=1e-3, mix_mode="both")
trainer.fit(num_epochs=4)

Trainer device: cuda
Epoch 1/4 | train_loss: 0.5294 | val_loss: 0.3499 | val_ROC_AUC: 0.9868
Epoch 2/4 | train_loss: 0.4617 | val_loss: 0.3460 | val_ROC_AUC: 0.9865
Epoch 3/4 | train_loss: 0.5406 | val_loss: 0.3446 | val_ROC_AUC: 0.9866
Epoch 4/4 | train_loss: 0.5225 | val_loss: 0.3283 | val_ROC_AUC: 0.9898
Best val ROC AUC: 0.9898
